# **1. Installing Library**

In [ ]:
!pip install twint

     |████████████████████████████████| 1.2MB 5.0MB/s 
     |████████████████████████████████| 245kB 13.0MB/s 
     |████████████████████████████████| 194kB 13.0MB/s 
     |████████████████████████████████| 153kB 14.8MB/s 
     |████████████████████████████████| 256kB 16.3MB/s 
     |████████████████████████████████| 235kB 12.4MB/s 
  Created wheel for twint: filename=twint-2.1.20-cp36-none-any.whl size=33921 sha256=a05da9aa6bb343aae716e99fce1a50684450414b8ab0e5bda288483508cd1f00
  Stored in directory: /root/.cache/pip/wheels/66/25/c7/855c1d896087ef84df6e6713b8adb073aff99af119450e1e4f
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=52ea8ddaf4f6b6baf0b11db6b44b16ecd6e65eb934f75ab2c714d1d96a4ff2e3
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for googletransx: filename=googletransx-2.4.2-cp36-none-any.whl size=15969 sha256=cf426366459c5e854e001a8bef0d7177029

In [ ]:
!pip install xlsxwriter

     |████████████████████████████████| 143kB 2.8MB/s 


# **2. Define Function**

In [ ]:
import twint
import pandas as pd
import time
from time import sleep
import asyncio
from datetime import datetime, timedelta
import os

## Fungsi Konversi Detik


In [ ]:
# function to display time more elegantly
intervals = (
    ('weeks', 604800),  # 60 * 60 * 24 * 7
    ('days', 86400),    # 60 * 60 * 24
    ('hours', 3600),    # 60 * 60
    ('minutes', 60),
    ('seconds', 1),
    )

def display_time(seconds, granularity=2):
    result = []

    for name, count in intervals:
        value = seconds // count
        if value:
            seconds -= value * count
            if value == 1:
                name = name.rstrip('s')
            result.append("{} {}".format(value, name))
    return ', '.join(result[:granularity])

## Fungsi Crowling Berdasarkan tanggal

In [ ]:
def by_date(query, since, until):
  query = query

  start_str = since
  end_str = until
  start_date = pd.to_datetime(start_str, format='%Y-%m-%d', errors='ignore')
  end_date = pd.to_datetime(end_str, format='%Y-%m-%d', errors='ignore')
  data_folder = "/content/" #Directory di sesuaikan dengan kebutuhan (Bisa di arahkan ke gdrive jika di run di colab)
  filename = f"{data_folder}{query}_tweets_{start_str}_{end_str}.csv"
  resume_file = f"{data_folder}resume.txt"

  c = twint.Config()
  c.Hide_output = True
  c.Store_csv = True
  c.Filter_retweets = True
  c.Custom["tweet"] = ['id', 'date', 'time','tweet','username']
  c.Output = filename
  c.Resume = resume_file
  c.Search = query
  c.Lang = 'id'

  while start_date < end_date:

      check = 0
      c.Since = datetime.strftime(start_date, format='%Y-%m-%d')
      c.Until = datetime.strftime(start_date + timedelta(days=1), format='%Y-%m-%d')

      while check < 1:
          try:
              print("Running Search: Check ", start_date)
              twint.run.Search(c)
              check += 1

          except Exception as e:
              print(e, "Sleeping for 7 mins")
              print("Check: ", check)
              sleep(420)

      os.remove(resume_file)

      start_date = start_date + timedelta(days=1)
  return filename

## Fungsi Crawling Berdasarkan Limit

In [ ]:
def by_limit(query, limit):
  query = query

  data_folder = "/content/" #Directory di sesuaikan dengan kebutuhan (Bisa di arahkan ke gdrive jika di run di colab)
  filename = f"{data_folder}{query}_tweets_{limit}.csv"
  resume_file = f"{data_folder}resume.txt"

  c = twint.Config()
  c.Hide_output = True
  c.Store_csv = True
  c.Filter_retweets = True
  c.Search = query
  c.Limit = limit
  c.Custom["tweet"] = ['id', 'date', 'time','tweet','username']
  c.Output = filename
  c.Resume = resume_file
  c.Lang = 'id'

  check = 0
  while check < 1:
      try:
          twint.run.Search(c)
          check += 1

      except Exception as e:
          print(e, "Sleeping for 7 mins")
          print("Check: ", check)
          sleep(420)

  os.remove(resume_file)

  return filename

## Fungsi Cleaning Tweet

In [ ]:
import re
from bs4 import BeautifulSoup
import lxml
from nltk.tokenize import WordPunctTokenizer

def tweet_cleaner(text):
  tok = WordPunctTokenizer()
  pat1 = r'@[A-Za-z0-9]+'
  pat2 = r'\w+:\/\/\S+'
  pat3 = r'\S*twitter.com\S*' 
  combined_pat = r'|'.join((pat1, pat2, pat3))
  soup = BeautifulSoup(text, 'lxml')
  souped = soup.get_text()
  stripped = re.sub(combined_pat, '', souped)
  try:
    clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
  except:
    clean = stripped
  letters_only = re.sub("[^a-zA-Z0-9]", " ", clean)
  lower_case = letters_only.lower()
  words = tok.tokenize(lower_case)
  return (" ".join(words)).strip()

## Gabungkan Semua Jadi satu fungsi

In [ ]:
def crawling_by_date(query, since, until, clean=False):
  """
  Dapat menggunakan lebih dari satu kata kunci 
  contoh "BNI OR Nasabah OR ATM"
  contoh "BNI AND ATM"

  Param since dan until formatnya :
  "YYYY-MM-DD"
  contoh : "2020-06-01"

  Param clean secara default False,
  set True jika ingin tweet yang di hasilkan dibersihkan
  """
  start_time = time.time()
  result_path = by_date(query, since, until)
  df = pd.read_csv(result_path)
  df = df.astype(str)
  if clean:
    df['tweet'] = df['tweet'].apply(lambda text: tweet_cleaner(text))
  writer = pd.ExcelWriter(query+"_tweet_"+since+"_"+until+".xlsx", engine = 'xlsxwriter')
  df.to_excel(writer, index=False)
  writer.save()
  writer.close()
  os.remove(result_path)
  print("Exe Time : ", display_time(time.time() - start_time))

In [ ]:
def crawling_by_limit(query, limit, clean=False):
  """
  Dapat menggunakan lebih dari satu kata kunci 
  contoh "BNI OR Nasabah OR ATM"
  contoh "BNI AND ATM"

  Limit berupa Angka

  Param clean secara default False,
  set True jika ingin tweet yang di hasilkan dibersihkan
  """
  start_time = time.time()
  result_path = by_limit(query, limit)
  df = pd.read_csv(result_path)
  df = df.astype(str)
  if clean:
    df['tweet'] = df['tweet'].apply(lambda text: tweet_cleaner(text))
  writer = pd.ExcelWriter(query+"_tweet_"+str(limit)+".xlsx", engine = 'xlsxwriter')
  df.to_excel(writer, index=False)
  writer.save()
  writer.close()
  os.remove(result_path)
  print("Exe Time : ", display_time(time.time() - start_time))

# **3. Execute Function**

In [ ]:
crawling_by_date('bni','2020-05-01', '2020-05-02', clean=True)

Running Search: Check  2020-05-01 00:00:00
Exe Time :  54.0 seconds


In [ ]:
crawling_by_limit('bni', 100)

Exe Time :  4.0 seconds
